In [1]:
import pandas as pd
import numpy as np
import requests
import random

!pip install geopy
from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed')
print('Libraries imported.')

Requirement not upgraded as not directly required: geopy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'FE5Z4BE34BWUOWLLBGWVPOFGNN2SHHF13LNDJLKQT3AGKRMS'
CLIENT_SECRET ='KSSNOXFWS32B0A40M2O1J4N0TTDWAQXSVGZLHD43DZGOKQSG'
VERSION = '20180318'
LIMIT = 30

print('My credentials:')
print('ID:', CLIENT_ID)
print('Secret:', CLIENT_SECRET)

My credentials:
ID: FE5Z4BE34BWUOWLLBGWVPOFGNN2SHHF13LNDJLKQT3AGKRMS
Secret: KSSNOXFWS32B0A40M2O1J4N0TTDWAQXSVGZLHD43DZGOKQSG


In [3]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
longitude = location.longitude
latitude = location.latitude
print(latitude, longitude)

40.7149555 -74.0153365


In [4]:
search_query='Italian'
radius = 500
print(search_query, '... OK!')

Italian ... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FE5Z4BE34BWUOWLLBGWVPOFGNN2SHHF13LNDJLKQT3AGKRMS&client_secret=KSSNOXFWS32B0A40M2O1J4N0TTDWAQXSVGZLHD43DZGOKQSG&ll=40.7149555,-74.0153365&v=20180318&query=Italian&radius=500&limit=30'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8fb7ad6a607168d039bae7'},
 'response': {'venues': [{'beenHere': {'count': 0,
     'lastCheckinExpiredAt': 0,
     'marked': False,
     'unconfirmedCount': 0},
    'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'primary': True,
      'shortName': 'Pizza'}],
    'contact': {},
    'delivery': {'id': '294544',
     'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
       'prefix': 'https://fastly.4sqi.net/img/general/cap/',
       'sizes': [40, 50]},
      'name': 'seamless'},
     'url': 'https://www.seamless.com/menu/harrys-italian-pizza-bar-225-murray-st-new-york/294544?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=294544'},
    'hasPerk': False,
    'id': '4fa862b3e4b0ebff2f749f06',
    

In [7]:
venues = results['response']['venues']

df = json_normalize(venues)
df.head(5)

,beenHere.count,beenHere.lastCheckinExpiredAt,beenHere.marked,beenHere.unconfirmedCount,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,...,location.postalCode,location.state,name,referralId,stats.checkinsCount,stats.tipCount,stats.usersCount,stats.visitsCount,venueChains,verified
0,0,0,False,0,"[{'shortName': 'Pizza', 'primary': True, 'name...",294544,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,...,10282,NY,Harry's Italian Pizza Bar,v-1552922541,0,0,0,0,[],False
1,0,0,False,0,"[{'shortName': 'Food', 'primary': True, 'name'...",NaN,NaN,NaN,NaN,NaN,...,10007,NY,Conca Cucina Italian Restaurant,v-1552922541,0,0,0,0,[],False
2,0,0,False,0,"[{'shortName': 'Italian', 'primary': True, 'na...",NaN,NaN,NaN,NaN,NaN,...,10007,NY,Ecco,v-1552922541,0,0,0,0,[],False


In [8]:
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = df.loc[:, filtered_columns]

In [9]:
dataframe_filtered

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,id
0,Harry's Italian Pizza Bar,"[{'shortName': 'Pizza', 'primary': True, 'name...",225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'lat': 40.71521779064671, 'label': 'display'...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,"[{'shortName': 'Food', 'primary': True, 'name'...",63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'lat': 40.71446, 'label': 'display', 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,"[{'shortName': 'Italian', 'primary': True, 'na...",124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'lat': 40.71533713859952, 'label': 'display'...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'lat': 40.71521779064671, 'label': 'display'...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'lat': 40.71446, 'label': 'display', 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'lat': 40.71533713859952, 'label': 'display'...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [11]:
dataframe_filtered.name

0          Harry's Italian Pizza Bar
1    Conca Cucina Italian Restaurant
2                               Ecco
Name: name, dtype: object

In [38]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [13]:
# explore Harry's Italian Pizza Bar

venue = dataframe_filtered[['name','id']].loc[dataframe_filtered['name']=="Harry's Italian Pizza Bar"]
venue

,name,id
0,Harry's Italian Pizza Bar,4fa862b3e4b0ebff2f749f06


In [14]:
venue_id = dataframe_filtered['id'].loc[dataframe_filtered['name']=="Harry's Italian Pizza Bar"]
venue_id=venue_id[0]
venue_id

'4fa862b3e4b0ebff2f749f06'

In [15]:
venue_id='4fa862b3e4b0ebff2f749f06'

In [16]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url_2 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url_2

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=FE5Z4BE34BWUOWLLBGWVPOFGNN2SHHF13LNDJLKQT3AGKRMS&client_secret=KSSNOXFWS32B0A40M2O1J4N0TTDWAQXSVGZLHD43DZGOKQSG&v=20180318'

In [17]:
result_2=requests.get(url_2).json()
print(result_2['response']['venue'].keys())
result_2['response']['venue']

dict_keys(['canonicalUrl', 'inbox', 'rating', 'bestPhoto', 'delivery', 'stats', 'venueChains', 'likes', 'ratingSignals', 'hasMenu', 'createdAt', 'menu', 'url', 'specials', 'ok', 'tips', 'hours', 'reasons', 'colors', 'allowMenuUrlEdit', 'hereNow', 'price', 'ratingColor', 'photos', 'dislike', 'popular', 'id', 'attributes', 'verified', 'pageUpdates', 'listed', 'categories', 'contact', 'beenHere', 'timeZone', 'name', 'location', 'shortUrl'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$$', 'priceTier': 2}],
    'name': 'Price',
    'summary': '$$',
    'type': 'price'},
   {'count': 7,
    'items': [{'displayName': 'Credit Cards',
      'displayValue': 'Yes (incl. American Express)'}],
    'name': 'Credit Cards',
    'summary': 'Credit Cards',
    'type': 'payments'},
   {'count': 1,
    'items': [{'displayName': 'Outdoor Seating', 'displayValue': 'Yes'}],
    'name': 'Outdoor Seating',
    'summary': 'Outdoor Seating',
    'type': 'outdoorSeating'},
   {'count': 8,
    'items': [{'displayName': 'Brunch', 'displayValue': 'Brunch'},
     {'displayName': 'Lunch', 'displayValue': 'Lunch'},
     {'displayName': 'Dinner', 'displayValue': 'Dinner'},
     {'displayName': 'Happy Hour', 'displayValue': 'Happy Hour'}],
    'name': 'Menus',
    'summary': 'Happy Hour, Brunch & more',
    'type': 'serves'},
   {'count': 5,
    'items': [{'displayName': 'Bee

In [18]:
result_2['response']['venue']['name']

"Harry's Italian Pizza Bar"

In [19]:
try:
    print(result_2['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.1


In [20]:
# id of Conca Cucina
venue_id_2 = dataframe_filtered['id'].loc[dataframe_filtered['name']=='Conca Cucina Italian Restaurant']
venue_id_2 = venue_id_2[1]
venue_id_2

'4f3232e219836c91c7bfde94'

In [21]:
url_3 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.\
format(venue_id_2, CLIENT_ID, CLIENT_SECRET, VERSION)
url_3

'https://api.foursquare.com/v2/venues/4f3232e219836c91c7bfde94?&client_id=FE5Z4BE34BWUOWLLBGWVPOFGNN2SHHF13LNDJLKQT3AGKRMS&client_secret=KSSNOXFWS32B0A40M2O1J4N0TTDWAQXSVGZLHD43DZGOKQSG&v=20180318'

In [22]:
result_3 = requests.get(url_3).json()
print(result_3['response']['venue'].keys())
result_3['response']['venue']

dict_keys(['canonicalUrl', 'reasons', 'stats', 'likes', 'hereNow', 'tips', 'id', 'timeZone', 'ok', 'venueChains', 'allowMenuUrlEdit', 'inbox', 'photos', 'dislike', 'createdAt', 'attributes', 'verified', 'pageUpdates', 'listed', 'categories', 'contact', 'beenHere', 'specials', 'name', 'location', 'shortUrl'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': []},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'canonicalUrl': 'https://foursquare.com/v/conca-cucina-italian-restaurant/4f3232e219836c91c7bfde94',
 'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
    'suffix': '.png'},
   'id': '4d4b7105d754a06374d81259',
   'name': 'Food',
   'pluralName': 'Food',
   'primary': True,
   'shortName': 'Food'}],
 'contact': {'formattedPhone': '(212) 732-5627', 'phone': '2127325627'},
 'createdAt': 1328689890,
 'dislike': False,
 'hereNow': {'count': 0, 'groups': [], 'summary': 'Nobody here'},
 'id': '4f3232e219836c91c7bfde94',
 'inbox': {'count': 0, 'items': []},
 'likes': {'count': 0, 'groups': []},
 'listed': {'count': 0,
  'groups': [{'count': 0,
    'items': [],
    'name': 'Lists from other people',
    'type': 'others'}]},
 'location': {'address': '63 W Broadway',
  'cc': 'US',
  'city': 'New Yo

In [23]:
try:
    print(result_3['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [24]:
venue_id_3 = '3fd66200f964a520f4e41ee3' # ID of Ecco
url_4 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id_3, CLIENT_ID, CLIENT_SECRET, VERSION)

result_4 = requests.get(url_4).json()
try:
    print(result_4['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.8


In [25]:
#Ecco has the best rating

result_4['response']['venue']['tips']['count']

16

In [26]:
# what are those tips?

url_tips = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit=15'.\
format(venue_id_3, CLIENT_ID, CLIENT_SECRET, VERSION)
url_tips

'https://api.foursquare.com/v2/venues/3fd66200f964a520f4e41ee3/tips?client_id=FE5Z4BE34BWUOWLLBGWVPOFGNN2SHHF13LNDJLKQT3AGKRMS&client_secret=KSSNOXFWS32B0A40M2O1J4N0TTDWAQXSVGZLHD43DZGOKQSG&v=20180318&limit=15'

In [27]:
result_tips = requests.get(url_tips).json()
result_tips

{'meta': {'code': 200, 'requestId': '5c8fb7b04c1f6747285bfdf3'},
 'response': {'tips': {'count': 16,
   'items': [{'agreeCount': 2,
     'authorInteractionType': 'liked',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'disagreeCount': 0,
     'id': '5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'todo': {'count': 0},
     'type': 'user',
     'user': {'firstName': 'Nick',
      'gender': 'male',
      'id': '484542633',
      'lastName': 'El-Tawil',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'}}}]}}}

In [28]:
tips = result_tips['response']['tips']['items']
tips

[{'agreeCount': 2,
  'authorInteractionType': 'liked',
  'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
  'createdAt': 1521601350,
  'disagreeCount': 0,
  'id': '5ab1cb46c9a517174651d3fe',
  'lang': 'en',
  'likes': {'count': 0, 'groups': []},
  'logView': True,
  'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
  'todo': {'count': 0},
  'type': 'user',
  'user': {'firstName': 'Nick',
   'gender': 'male',
   'id': '484542633',
   'lastName': 'El-Tawil',
   'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
    'suffix': '/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'}}}]

In [29]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",2,0,5ab1cb46c9a517174651d3fe,Nick,El-Tawil,male,484542633


In [30]:
# searching for other reviews of this user
user_id = tips_filtered['user.id'].loc[tips_filtered['user.firstName']=='Nick']
user_id=user_id[0]
user_id

'484542633'

In [31]:
user_url= 'https://api.foursquare.com/v2/users/{}?&client_id={}&client_secret={}&v={}'.\
format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION)

results_user = requests.get(user_url).json()
user_data = results_user['response']['user']

user_data.keys()


dict_keys(['canonicalUrl', 'type', 'bio', 'photos', 'checkins', 'photo', 'friends', 'mayorships', 'firstName', 'lenses', 'homeCity', 'gender', 'lastName', 'contact', 'tips', 'id', 'lists'])

In [32]:
print('First name: ' + user_data['firstName'])
print('Last name: ' + user_data['lastName'])
print('Home city: ' + user_data['homeCity'])

First name: Nick
Last name: El-Tawil
Home city: New York, NY


In [33]:
user_data['tips']

{'count': 240}

In [34]:
# define tips URL
url_5 = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit=15'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET request and get user's tips
results_6 = requests.get(url_5).json()
tips = results_6['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,"The best! I’m especially fond of the salmon burger, but I’ve had half of the menu and never been disappointed. There’s a reason this place is well known even outside of the Village!",1,0,5aec594b1f7440002c138612
1,I used to down a pint of chocolate like it was nothing back when I was bulking. Highly recommended!,1,0,5accc9f66fa81f196724807b
2,They serve coffee!!!!!!,1,0,5accc98c0313204c9d7ec157
3,"I’m a fan. In fact, I’m such a big fan, I want Taim to hire me to be their spokesman. Kind of like the Arabic Jared Fogle - but without the kid stuff.",1,0,5accbf033abcaf09a24612a0
4,The linguine with clams is on point 👌,1,0,5accbe3a911fc423730f3ed3
5,"Great for a quick, cheap lunch! Shorter lines than Chipotle too👌",1,0,5acbecb86fa81f1967e019b0
6,"Quick, cheap lunch that tastes good! Way shorter line than Chipotle, too.",1,0,5acbec70a0215b732e264fe8
7,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
8,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,2,0,5acbbcda01235808d5d6dc75
9,Coffee game on point,1,0,5acbbb1501235808d5d6525e


In [35]:
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url_8 = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result_8 = requests.get(url_8).json()
print(result_8['response']['tip']['venue']['name'])
print(result_8['response']['tip']['venue']['location'])

Cowgirl
{'state': 'NY', 'country': 'United States', 'labeledLatLngs': [{'lat': 40.73373338282062, 'label': 'display', 'lng': -74.0062998849649}], 'crossStreet': 'at W 10th St', 'address': '519 Hudson St', 'postalCode': '10014', 'lat': 40.73373338282062, 'formattedAddress': ['519 Hudson St (at W 10th St)', 'New York, NY 10014', 'United States'], 'city': 'New York', 'lng': -74.0062998849649, 'cc': 'US'}


In [36]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

""


In [37]:
user_data

{'bio': 'https://www.tawil.team/nick-el-tawil/',
 'canonicalUrl': 'https://foursquare.com/nickeltawil',
 'checkins': {'count': 1, 'items': []},
 'contact': {},
 'firstName': 'Nick',
 'friends': {'count': 0,
  'groups': [{'count': 0,
    'items': [],
    'name': 'Other friends',
    'type': 'others'}]},
 'gender': 'male',
 'homeCity': 'New York, NY',
 'id': '484542633',
 'lastName': 'El-Tawil',
 'lenses': [],
 'lists': {'count': 2,
  'groups': [{'count': 0, 'items': [], 'type': 'created'},
   {'count': 0, 'items': [], 'type': 'followed'},
   {'count': 2,
    'items': [{'canonicalUrl': 'https://foursquare.com/nickeltawil/list/todos',
      'collaborative': False,
      'description': '',
      'editable': False,
      'id': '484542633/todos',
      'listItems': {'count': 0},
      'name': "Nick's Saved Places",
      'public': True,
      'type': 'todos',
      'url': '/nickeltawil/list/todos'},
     {'canonicalUrl': 'https://foursquare.com/nickeltawil/list/venuelikes',
      'collaborat

In [43]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://igx.4sqi.net/img/user/300x300/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg')

In [46]:
# explore a random location
latitude = 40.715337
longitude = -74.008848

url_9 = 'http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit={}'.\
format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, LIMIT)
url_9

'http://api.foursquare.com/v2/venues/explore?client_id=FE5Z4BE34BWUOWLLBGWVPOFGNN2SHHF13LNDJLKQT3AGKRMS&client_secret=KSSNOXFWS32B0A40M2O1J4N0TTDWAQXSVGZLHD43DZGOKQSG&ll=40.715337,-74.008848&v=20180318&limit=30'

In [47]:
results = requests.get(url_9).json()
results

{'meta': {'code': 200, 'requestId': '5c8fbbfaf594df7ce03888d4'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4af5d65ff964a52091fd21e3-0',
      'venue': {'beenHere': {'count': 0,
        'lastCheckinExpiredAt': 0,
        'marked': False,
        'unconfirmedCount': 0},
       'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/furniture_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1f8941735',
         'name': 'Furniture / Home Store',
         'pluralName': 'Furniture / Home Stores',
         'primary': True,
         'shortName': 'Furniture / Home'}],
       'contact': {},
       'hereNow': {'count': 0, 'groups': [], 'summary': 'Nobody here'},
       'id': '4af5d65ff964a52091fd21e3',
       'location': {'address': '57 Warren St',
        'cc': 'US',
        'city':

In [48]:
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

In [49]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4af5d65ff964a52091fd21e3-0',
 'venue': {'beenHere': {'count': 0,
   'lastCheckinExpiredAt': 0,
   'marked': False,
   'unconfirmedCount': 0},
  'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/furniture_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1f8941735',
    'name': 'Furniture / Home Store',
    'pluralName': 'Furniture / Home Stores',
    'primary': True,
    'shortName': 'Furniture / Home'}],
  'contact': {},
  'hereNow': {'count': 0, 'groups': [], 'summary': 'Nobody here'},
  'id': '4af5d65ff964a52091fd21e3',
  'location': {'address': '57 Warren St',
   'cc': 'US',
   'city': 'New York',
   'country': 'United States',
   'crossStreet': 'Church St',
   'distance': 73,
   'formattedAddress': ['57 Warren St (Church St)',
    'New York, NY 10007',
    'United States'],
   'labeledL

In [50]:
dataframe = json_normalize(items) # flatten JSON
dataframe.head()

,reasons.count,reasons.items,referralId,venue.beenHere.count,venue.beenHere.lastCheckinExpiredAt,venue.beenHere.marked,venue.beenHere.unconfirmedCount,venue.categories,venue.delivery.id,venue.delivery.provider.icon.name,...,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.stats.checkinsCount,venue.stats.tipCount,venue.stats.usersCount,venue.stats.visitsCount,venue.venuePage.id,venue.verified
0,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",e-0-4af5d65ff964a52091fd21e3-0,0,0,False,0,"[{'shortName': 'Furniture / Home', 'primary': True, 'name': 'Furniture / Home Store', 'id': '4bf58dd8d48988d1f8941735', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/furniture_', 'suffix': '.png'}, 'pluralName': 'Furniture / Home Stores'}]",NaN,NaN,...,NY,Korin,0,[],0,0,0,0,33104775,True
1,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",e-0-4adcf23cf964a520cc6221e3-1,0,0,False,0,"[{'shortName': 'Wine Shop', 'primary': True, 'name': 'Wine Shop', 'id': '4bf58dd8d48988d119951735', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_wineshop_', 'suffix': '.png'}, 'pluralName': 'Wine Shops'}]",NaN,NaN,...,NY,Chambers Street Wines,0,[],0,0,0,0,NaN,False
2,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",e-0-4c154c9a77cea593c401d260-2,0,0,False,0,"[{'shortName': 'Bakery', 'primary': True, 'name': 'Bakery', 'id': '4bf58dd8d48988d16a941735', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_', 'suffix': '.png'}, 'pluralName': 'Bakeries'}]",NaN,NaN,...,NY,Takahachi Bakery,0,[],0,0,0,0,85495566,True
3,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",e-0-4a8f2f39f964a520471420e3-3,0,0,False,0,"[{'shortName': 'Sushi', 'primary': True, 'name': 'Sushi Restaurant', 'id': '4bf58dd8d48988d1d2941735', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_', 'suffix': '.png'}, 'pluralName': 'Sushi Restaurants'}]",NaN,NaN,...,NY,Takahachi,0,[],0,0,0,0,NaN,False
4,0,"[{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]",e-0-54148bc6498ea7bb8c05b70a-4,0,0,False,0,"[{'shortName': 'Vegetarian / Vegan', 'primary': True, 'name': 'Vegetarian / Vegan Restaurant', 'id': '4bf58dd8d48988d1d3941735', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_', 'suffix': '.png'}, 'pluralName': 'Vegetarian / Vegan Restaurants'}]",NaN,NaN,...,NY,Juice Press,0,[],0,0,0,0,NaN,True


In [51]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Korin,Furniture / Home Store,57 Warren St,US,New York,United States,Church St,73,"[57 Warren St (Church St), New York, NY 10007, United States]","[{'lat': 40.71482437714839, 'label': 'display', 'lng': -74.00940425461492}]",40.714824,-74.009404,Tribeca,10007,NY,4af5d65ff964a52091fd21e3
1,Chambers Street Wines,Wine Shop,148 Chambers St,US,New York,United States,btwn West Broadway & Hudson St,88,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]","[{'lat': 40.715773063928374, 'label': 'display', 'lng': -74.00971823312332}]",40.715773,-74.009718,NaN,10007,NY,4adcf23cf964a520cc6221e3
2,Takahachi Bakery,Bakery,25 Murray St,US,New York,United States,at Church St,187,"[25 Murray St (at Church St), New York, NY 10007, United States]","[{'lat': 40.713652845301894, 'label': 'display', 'lng': -74.0088038953017}]",40.713653,-74.008804,NaN,10007,NY,4c154c9a77cea593c401d260
3,Takahachi,Sushi Restaurant,145 Duane St,US,New York,United States,btwn W Broadway & Church St,146,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]","[{'lat': 40.71652647412374, 'label': 'display', 'lng': -74.00810108466207}]",40.716526,-74.008101,NaN,10013,NY,4a8f2f39f964a520471420e3
4,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,US,New York,United States,btwn Greenwich St & W Broadway,202,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]","[{'lat': 40.71478769908051, 'label': 'display', 'lng': -74.0111317502157}]",40.714788,-74.011132,NaN,10007,NY,54148bc6498ea7bb8c05b70a
5,Nish Nūsh,Falafel Restaurant,88 Reade St,US,New York,United States,at Church St,97,"[88 Reade St (at Church St), New York, NY 10013, United States]","[{'lat': 40.71553710116416, 'label': 'display', 'lng': -74.00772452925565}]",40.715537,-74.007725,NaN,10013,NY,50ba9119e4b071a4bae6dc10
6,Mulberry & Vine,Café,73 Warren St,US,New York,United States,btwn W Broadway & Greenwich St,117,"[73 Warren St (btwn W Broadway & Greenwich St), New York, NY 10007, United States]","[{'lat': 40.71517693966315, 'label': 'display', 'lng': -74.01022747778285}]",40.715177,-74.010227,NaN,10007,NY,5171b5cc011cef9833bbb787
7,Equinox Tribeca,Gym,54 Murray Street,US,New York,United States,at W Broadway,154,"[54 Murray Street (at W Broadway), New York, NY 10007, United States]","[{'lat': 40.71409860726041, 'label': 'display', 'lng': -74.0096857179283}]",40.714099,-74.009686,NaN,10007,NY,4a6e331af964a52031d41fe3
8,Philip Williams Posters,Antique Shop,122 Chambers St,US,New York,United States,NaN,8,"[122 Chambers St, New York, NY 10007, United States]","[{'lat': 40.71528423132827, 'label': 'display', 'lng': -74.00878093952018}]",40.715284,-74.008781,NaN,10007,NY,4b747291f964a52042dd2de3
9,Whole Foods Market,Grocery Store,270 Greenwich Street,US,New York,United States,at Warren St,214,"[270 Greenwich Street (at Warren St), New York, NY 10007, United States]","[{'lat': 40.715579155420606, 'label': 'display', 'lng': -74.01136823958119}]",40.715579,-74.011368,Tribeca,10007,NY,49bc3b0af964a52020541fe3


In [52]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [58]:
#what are the trending venues in that area?
url_10 = 'http://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.\
format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, LIMIT)
url_10

'http://api.foursquare.com/v2/venues/trending?client_id=FE5Z4BE34BWUOWLLBGWVPOFGNN2SHHF13LNDJLKQT3AGKRMS&client_secret=KSSNOXFWS32B0A40M2O1J4N0TTDWAQXSVGZLHD43DZGOKQSG&ll=40.715337,-74.008848&v=20180318'

In [59]:
results_10 = requests.get(url_10).json()
results_10

{'meta': {'code': 200, 'requestId': '5c8fc45c6a607168d53c908e'},
 'response': {'venues': []}}

In [63]:
if len(results_10['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]
    
    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)                                                

In [64]:
trending_venues_df

'No trending venues are available at the moment!'

In [66]:
if len(results_10['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [67]:
venues_map

'Cannot generate visual as no trending venues are available at the moment!'